## Functionality that needs to be added

- ~~Trim Raw Input~~
- Night vs Day bouts (critera - choose where over 50% of bout is spent, in case of tie choose night)
- Create Sleep Profile and summary stats
- Write to seperate sheets in Excel

### Import Libraries that will be needed

In [85]:
import pandas as pd
import datetime

### Import data from .csv

In [2]:
df = pd.read_csv("BethTestData.csv")

In [12]:
# Trim to appropriate length for 24h time window
df = df[:288]

### Trim to 24 hour range

In [102]:
def get_data_range(L):
    """
    Find the start and end indices for a 24 hour range starting between 
    10:00:00 and 10:04:59 and ending 24 hours later
    *** Assumes the first instance of time between 10:00:00 and 10:04:59 is the start time! ***
    Takes arguments L - a list of datetime.time objects
    Returns a tuple: (start_index, end_index)
    """
    lower_limit = datetime.time(10,0,0)
    upper_limit = datetime.time(10,5,0)
    start_index = 0
    for i in range(len(L)):
        if L[i] >= lower_limit and L[i] < upper_limit:
            start_index = i
            break
    return (start_index, start_index + 288)


In [104]:
# Test trim on raw data
df_raw = pd.read_csv("Bethany_Raw.csv")
df_raw.Time = pd.to_datetime(df_raw.Time, format='%H:%M:%S').dt.time

In [105]:
trim_index = get_data_range(list(df_raw.Time))
df_trim = df_raw[trim_index[0]:trim_index[1]]

In [106]:
df_trim.tail()

,Time,Filename,mean_Deg_C_1_1,uL_BL,uL_Channel_2,uL_Channel_3,uL_Channel_4,uL_Channel_5,uL_Channel_6,Unnamed: 9,Baseline,w1118 BL 6326,w1118 BL 6326.1,eaat2 e04636,eaat2 e04636.1,eaat2 e04636.2
548,09:36:06,RepRec~04-28-2017_0552.exp,25.11143,0.169214,1.234814,0.955284,0.845982,0.881052,1.299192,NaN,0.0,0,0,0,0,0.0
549,09:41:06,RepRec~04-28-2017_0553.exp,24.85724,0.180109,1.185296,0.846394,0.833460,0.932308,1.324043,NaN,0.0,0,0,0,0,0.0
550,09:46:06,RepRec~04-28-2017_0554.exp,24.68118,0.171381,1.140252,0.803339,0.805544,0.907936,1.284124,NaN,0.0,0,0,0,0,0.0
551,09:51:06,RepRec~04-28-2017_0555.exp,24.54192,0.160445,1.135841,0.847280,0.801460,0.887825,1.273281,NaN,0.0,0,0,0,0,0.0
552,09:56:06,RepRec~04-28-2017_0556.exp,24.42790,0.168271,1.144841,0.839181,0.801850,0.896463,1.269652,NaN,0.0,0,0,0,0,0.0


In [18]:
# Need to add col for identity of night vs day

### Generate bout indices

In [30]:
# Script for collecting bouts
# Will need to create a list of bout indices to access resp data during bouts
def get_bout_indices(L):
    """
    Takes a list L and returns a list of tuples of the start/end indices in which sleeping bouts occurred.
    I.e. if two sleeping bouts occured, the first from index 5 to 20, and the second from index 30 to 40,
    this function will return [(5,20), (30,40)]
    """
    indices = []
    start_index = 1
    end_index = 1
    in_bout = False
    
    for i in range(len(L)):
        if L[i] == 0 and in_bout == False: 
            start_index = i
            in_bout = True
        if (L[i] != 0 or i == len(L)-1) and in_bout == True:
            end_index = i
            in_bout = False
            indices.append((start_index, end_index))
    return indices 
        

In [59]:
# Will now need to access corresponding respirometry values in df, and store values in a list
bout_indices = get_bout_indices(list(df["w1118 BL 6326"]))
resp_bouts = []
for i in bout_indicies:
    resp_bouts.append(list(df.uL_Channel_2)[i[0]:i[1]])

## Export to .csv

In [51]:
# Now we put this list of lists into a csv
resp_bouts_df = pd.DataFrame(resp_bouts).transpose()
resp_bouts_df.to_csv('test.csv', index=False, header=False)